# Espaciado

In [2]:
import os
import SimpleITK as sitk
import numpy as np

base_dir = "../../../data/images"

t2w_spacings = []
adc_spacings = []
hbv_spacings = []

for patient_id in os.listdir(base_dir):
    patient_path = os.path.join(base_dir, patient_id)
    
    if not os.path.isdir(patient_path):
        continue
    
    for file_name in os.listdir(patient_path):
        if file_name.endswith(".mha"):
            image_path = os.path.join(patient_path, file_name)
            
            if "t2w" in file_name.lower():
                img = sitk.ReadImage(image_path)
                t2w_spacings.append(img.GetSpacing())
            elif "adc" in file_name.lower():
                img = sitk.ReadImage(image_path)
                adc_spacings.append(img.GetSpacing())
            elif "hbv" in file_name.lower():
                img = sitk.ReadImage(image_path)
                hbv_spacings.append(img.GetSpacing())

def print_mean_spacing(name, spacings):
    if len(spacings) > 0:
        arr = np.array(spacings)
        mean_spac = arr.mean(axis=0)
        print(f"{name} - Mean spacing: {mean_spac} (n={len(spacings)})")
    else:
        print(f"No se encontraron imágenes para la modalidad {name}.")

print_mean_spacing("T2W", t2w_spacings)
print_mean_spacing("ADC", adc_spacings)
print_mean_spacing("HBV", hbv_spacings)

all_spacings = t2w_spacings + adc_spacings + hbv_spacings
if all_spacings:
    print_mean_spacing("Global", all_spacings)

T2W - Mean spacing: [0.42174294 0.42174294 3.19030266] (n=1500)
ADC - Mean spacing: [1.85423755 1.85423755 3.25136167] (n=1500)
HBV - Mean spacing: [1.85423755 1.85423755 3.25862799] (n=1500)
Global - Mean spacing: [1.37673935 1.37673935 3.23343077] (n=4500)


ModuleNotFoundError: No module named 'radiomics'

In [9]:
adc_ranges = [] 

# Recorremos cada carpeta de paciente
for patient_id in os.listdir(base_dir):
    patient_path = os.path.join(base_dir, patient_id)
    if not os.path.isdir(patient_path):
        continue

    # Buscamos archivos que sean imágenes ADC (por ejemplo, terminadas en .mha y contengan "adc")
    for file_name in os.listdir(patient_path):
        if file_name.lower().endswith(".mha") and "adc" in file_name.lower():
            image_path = os.path.join(patient_path, file_name)
            try:
                image = sitk.ReadImage(image_path)
                # Crear una máscara completa (full mask) para cubrir todo el volumen
                image_array = sitk.GetArrayFromImage(image)
                mask_array = np.ones_like(image_array, dtype=np.uint8)
                mask = sitk.GetImageFromArray(mask_array)
                mask.CopyInformation(image)
                
                # Extraer las características con pyradiomics
                result = extractor.execute(image, mask)
                # Se espera que la clave sea "original_firstorder_Range"
                if "original_firstorder_Range" in result:
                    range_value = result["original_firstorder_Range"]
                    adc_ranges.append(range_value)
                else:
                    print(f"Imagen: {image_path} no contiene 'original_firstorder_Range' en el resultado.")
            except Exception as e:
                print(f"Error procesando {image_path}: {e}")

In [10]:
if adc_ranges:
    adc_ranges = np.array(adc_ranges)
    mean_range = np.mean(adc_ranges)
    min_range = np.min(adc_ranges)
    max_range = np.max(adc_ranges)
    print("\nEstadísticas de firstorder:Range en ADC:")
    print(f"  Media: {mean_range:.2f}")
    print(f"  Mínimo: {min_range:.2f}")
    print(f"  Máximo: {max_range:.2f}")
    
    # Sugerir un binWidth basado en la media del rango y distintos números objetivo de bins
    target_bins_list = [16, 32, 64, 128]
    print("\nSugerencia de binWidth (usando la media del firstorder:Range):")
    for tb in target_bins_list:
        suggested_bw = mean_range / tb
        print(f"  Para {tb} bins: binWidth = {suggested_bw:.2f}")
else:
    print("No se encontraron imágenes ADC o no se pudo extraer 'firstorder:Range'.")


Estadísticas de firstorder:Range en ADC:
  Media: 3921.11
  Mínimo: 2033.00
  Máximo: 5457.00

Sugerencia de binWidth (usando la media del firstorder:Range):
  Para 16 bins: binWidth = 245.07
  Para 32 bins: binWidth = 122.53
  Para 64 bins: binWidth = 61.27
  Para 128 bins: binWidth = 30.63


In [11]:
adc_ranges = [] 

# Recorremos cada carpeta de paciente
for patient_id in os.listdir(base_dir):
    patient_path = os.path.join(base_dir, patient_id)
    if not os.path.isdir(patient_path):
        continue

    # Buscamos archivos que sean imágenes ADC (por ejemplo, terminadas en .mha y contengan "adc")
    for file_name in os.listdir(patient_path):
        if file_name.lower().endswith(".mha") and "t2w" in file_name.lower():
            image_path = os.path.join(patient_path, file_name)
            try:
                image = sitk.ReadImage(image_path)
                # Crear una máscara completa (full mask) para cubrir todo el volumen
                image_array = sitk.GetArrayFromImage(image)
                mask_array = np.ones_like(image_array, dtype=np.uint8)
                mask = sitk.GetImageFromArray(mask_array)
                mask.CopyInformation(image)
                
                # Extraer las características con pyradiomics
                result = extractor.execute(image, mask)
                # Se espera que la clave sea "original_firstorder_Range"
                if "original_firstorder_Range" in result:
                    range_value = result["original_firstorder_Range"]
                    adc_ranges.append(range_value)
                else:
                    print(f"Imagen: {image_path} no contiene 'original_firstorder_Range' en el resultado.")
            except Exception as e:
                print(f"Error procesando {image_path}: {e}")

In [12]:
if adc_ranges:
    adc_ranges = np.array(adc_ranges)
    mean_range = np.mean(adc_ranges)
    min_range = np.min(adc_ranges)
    max_range = np.max(adc_ranges)
    print("\nEstadísticas de firstorder:Range en ADC:")
    print(f"  Media: {mean_range:.2f}")
    print(f"  Mínimo: {min_range:.2f}")
    print(f"  Máximo: {max_range:.2f}")
    
    # Sugerir un binWidth basado en la media del rango y distintos números objetivo de bins
    target_bins_list = [16, 32, 64, 128]
    print("\nSugerencia de binWidth (usando la media del firstorder:Range):")
    for tb in target_bins_list:
        suggested_bw = mean_range / tb
        print(f"  Para {tb} bins: binWidth = {suggested_bw:.2f}")
else:
    print("No se encontraron imágenes ADC o no se pudo extraer 'firstorder:Range'.")


Estadísticas de firstorder:Range en ADC:
  Media: 985.30
  Mínimo: 168.00
  Máximo: 4044.00

Sugerencia de binWidth (usando la media del firstorder:Range):
  Para 16 bins: binWidth = 61.58
  Para 32 bins: binWidth = 30.79
  Para 64 bins: binWidth = 15.40
  Para 128 bins: binWidth = 7.70


In [1]:
adc_ranges = [] 

# Recorremos cada carpeta de paciente
for patient_id in os.listdir(base_dir):
    patient_path = os.path.join(base_dir, patient_id)
    if not os.path.isdir(patient_path):
        continue

    # Buscamos archivos que sean imágenes ADC (por ejemplo, terminadas en .mha y contengan "adc")
    for file_name in os.listdir(patient_path):
        if file_name.lower().endswith(".mha") and "hbv" in file_name.lower():
            image_path = os.path.join(patient_path, file_name)
            try:
                image = sitk.ReadImage(image_path)
                # Crear una máscara completa (full mask) para cubrir todo el volumen
                image_array = sitk.GetArrayFromImage(image)
                mask_array = np.ones_like(image_array, dtype=np.uint8)
                mask = sitk.GetImageFromArray(mask_array)
                mask.CopyInformation(image)
                
                # Extraer las características con pyradiomics
                result = extractor.execute(image, mask)
                # Se espera que la clave sea "original_firstorder_Range"
                if "original_firstorder_Range" in result:
                    range_value = result["original_firstorder_Range"]
                    adc_ranges.append(range_value)
                else:
                    print(f"Imagen: {image_path} no contiene 'original_firstorder_Range' en el resultado.")
            except Exception as e:
                print(f"Error procesando {image_path}: {e}")

NameError: name 'os' is not defined

In [ ]:
if adc_ranges:
    adc_ranges = np.array(adc_ranges)
    mean_range = np.mean(adc_ranges)
    min_range = np.min(adc_ranges)
    max_range = np.max(adc_ranges)
    print("\nEstadísticas de firstorder:Range en ADC:")
    print(f"  Media: {mean_range:.2f}")
    print(f"  Mínimo: {min_range:.2f}")
    print(f"  Máximo: {max_range:.2f}")
    
    # Sugerir un binWidth basado en la media del rango y distintos números objetivo de bins
    target_bins_list = [16, 32, 64, 128]
    print("\nSugerencia de binWidth (usando la media del firstorder:Range):")
    for tb in target_bins_list:
        suggested_bw = mean_range / tb
        print(f"  Para {tb} bins: binWidth = {suggested_bw:.2f}")
else:
    print("No se encontraron imágenes ADC o no se pudo extraer 'firstorder:Range'.")